In [ ]:
import numpy as np
import pandas as pd

import chromadb
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

import yaml
CONFIG = yaml.safe_load(open("../config.yml"))
model = SentenceTransformerEmbeddings(model_name=CONFIG["EmbeddingModel"])

client = chromadb.PersistentClient(path=f"../{CONFIG['ChromaDBPath']}")
movie_collection = client.get_collection(name="movies", embedding_function=model)

In [ ]:
movie_collection.get(ids=["1032"])

In [ ]:
user_collection = client.get_collection(name="users", embedding_function=model)
user_collection.get(ids="7")

In [ ]:
from langchain.vectorstores import Chroma

langchain_chroma = Chroma(
    client=client,
    collection_name="movies",
    embedding_function=model,
)

In [ ]:
user_embedding = user_collection.get(ids="7", include=["embeddings"])["embeddings"]

langchain_chroma.similarity_search_by_vector_with_relevance_scores(user_embedding, k=3)

In [ ]:
movies = pd.read_csv("../data/movies.csv")
users = pd.read_csv("../data/user_history.csv")

In [ ]:
user_collection.get(ids="7")["metadatas"][0]["movieIds"]

In [ ]:
movies[movies.movieId.apply(int).apply(str).isin(['5991'])]